In [4]:
#1. Assam State Transport Corporation (ASRTC)

from selenium import webdriver  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        # AC_type = driver.find_elemets(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                # "Select_AC_Type" : AC_type[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('ASRTC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'ASRTC_bus_details.csv'")
    finally:
        driver.quit()


View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF66CA3FB05+28789]
	(No symbol) [0x00007FF66C9A86E0]
	(No symbol) [0x00007FF66C84592A]
	(No symbol) [0x00007FF66C89930E]
	(No symbol) [0x00007FF66C8995FC]
	(No symbol) [0x00007FF66C8E28A7]
	(No symbol) [0x00007FF66C8BF47F]
	(No symbol) [0x00007FF66C8DF654]
	(No symbol) [0x00007FF66C8BF1E3]
	(No symbol) [0x00007FF66C88A938]
	(No symbol) [0x00007FF66C88BAA1]
	GetHandleVerifier [0x00007FF66CD7933D+3410093]
	GetHandleVerifier [0x00007FF66CD8E7DD+3497293]
	GetHandleVerifier [0x00007FF66CD82A73+3448803]
	GetHandleVerifier [0x00007FF66CB07BBB+848171]
	(No symbol) [0x00007FF66C9B3C3F]
	(No symbol) [0x00007FF66C9AF6E4]
	(No symbol) [0x00007FF66C9AF87D]
	(No symbol) [0x00007FF66C99ED49]
	BaseThreadInitThunk [0x00007FFDE6AA259D+29]
	RtlUserThreadStart [0x00007FFDE760AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF66CA3FB05+28789]
	(No symbol) [0x00007FF66C9A86E0]
	(No symbol) [0

In [4]:
#2. WBTC (CTC)
 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('WBTC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'WBTC_bus_details.csv'")
    finally:
        driver.quit()


View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B9BBFB05+28789]
	(No symbol) [0x00007FF6B9B286E0]
	(No symbol) [0x00007FF6B99C592A]
	(No symbol) [0x00007FF6B9A1930E]
	(No symbol) [0x00007FF6B9A195FC]
	(No symbol) [0x00007FF6B9A628A7]
	(No symbol) [0x00007FF6B9A3F47F]
	(No symbol) [0x00007FF6B9A5F654]
	(No symbol) [0x00007FF6B9A3F1E3]
	(No symbol) [0x00007FF6B9A0A938]
	(No symbol) [0x00007FF6B9A0BAA1]
	GetHandleVerifier [0x00007FF6B9EF933D+3410093]
	GetHandleVerifier [0x00007FF6B9F0E7DD+3497293]
	GetHandleVerifier [0x00007FF6B9F02A73+3448803]
	GetHandleVerifier [0x00007FF6B9C87BBB+848171]
	(No symbol) [0x00007FF6B9B33C3F]
	(No symbol) [0x00007FF6B9B2F6E4]
	(No symbol) [0x00007FF6B9B2F87D]
	(No symbol) [0x00007FF6B9B1ED49]
	BaseThreadInitThunk [0x00007FF9A381259D+29]
	RtlUserThreadStart [0x00007FF9A486AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B9BBFB05+28789]
	(No symbol) [0x00007FF6B9B286E0]
	(No symbol) [0

In [5]:
#3. RSRTC Bus Routes & Timings

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('RSRTC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'RSRTC_bus_details.csv'")
    finally:
        driver.quit()


Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF66CA3FB05+28789]
	(No symbol) [0x00007FF66C9A86E0]
	(No symbol) [0x00007FF66C84592A]
	(No symbol) [0x00007FF66C89930E]
	(No symbol) [0x00007FF66C8995FC]
	(No symbol) [0x00007FF66C8E28A7]
	(No symbol) [0x00007FF66C8BF47F]
	(No symbol) [0x00007FF66C8DF654]
	(No symbol) [0x00007FF66C8BF1E3]
	(No symbol) [0x00007FF66C88A938]
	(No symbol) [0x00007FF66C88BAA1]
	GetHandleVerifier [0x00007FF66CD7933D+3410093]
	GetHandleVerifier [0x00007FF66CD8E7DD+3497293]
	GetHandleVerifier [0x00007FF66CD82A73+3448803]
	GetHandleVerifier [0x00007FF66CB07BBB+848171]
	(No symbol) [0x00007FF66C9B3C3F]
	(No symbol) [0x00007FF66C9AF6E4]
	(No symbol) [0x00007FF66C9AF87D]
	(No symbol) [0x00007FF66C99ED49]
	BaseThreadInitThunk [0x00007FFDE6AA259D+29]
	RtlUserThreadStart [0x00007FFDE760AF38+40]

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF66CA3FB05+28789]
	(No symbol) [0x00007FF66C9A8

In [5]:
#4. JKRTC Bus Routes & Timings

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        AC_type = driver.find_elemets(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Select_AC_Type" : AC_type[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/jksrtc"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('JKRTC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'JKRTC_bus_details.csv'")
    finally:
        driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-srinagar: 'WebDriver' object has no attribute 'find_elemets'
Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/jammu-to-katra: 'WebDriver' object has no attribute 'find_elemets'
Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/katra-to-jammu: 'WebDriver' object has no attribute 'find_elemets'
Error occurred while accessing page 2: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B9BBFB05+28789]
	(No symbol) [0x00007FF6B9B286E0]
	(No symbol) [0x00007FF6B99C592A]
	(No symbol) [0x00007FF6B9A1930E]
	(No symbol) [0x00007FF6B9A195FC]
	(No symbol) [0x00007FF6B9A628A7]
	(No symbol) [0x00007FF6B9A3F47F]
	(No symbol) [0x00007FF6B9A5F654]
	(No symbol) [0x00007FF6B9A3F1E3]
	(No symbol) [0x00007FF6B9A0A938]
	(No symbol) [0x00007FF6B9A0BAA1]
	GetHandleVerifier [0x00007FF6B9EF933D+3410093]
	GetHandleVerifier [0x00007FF6B9F0E7DD+3497293]
	GetHandleVer

In [6]:
#5. West bengal transport corporation Bus Routes & Timings

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('WBSTC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'WBSTC_bus_details.csv'")
    finally:
        driver.quit()


View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CB3DFB05+28789]
	(No symbol) [0x00007FF7CB3486E0]
	(No symbol) [0x00007FF7CB1E592A]
	(No symbol) [0x00007FF7CB23930E]
	(No symbol) [0x00007FF7CB2395FC]
	(No symbol) [0x00007FF7CB2828A7]
	(No symbol) [0x00007FF7CB25F47F]
	(No symbol) [0x00007FF7CB27F654]
	(No symbol) [0x00007FF7CB25F1E3]
	(No symbol) [0x00007FF7CB22A938]
	(No symbol) [0x00007FF7CB22BAA1]
	GetHandleVerifier [0x00007FF7CB71933D+3410093]
	GetHandleVerifier [0x00007FF7CB72E7DD+3497293]
	GetHandleVerifier [0x00007FF7CB722A73+3448803]
	GetHandleVerifier [0x00007FF7CB4A7BBB+848171]
	(No symbol) [0x00007FF7CB353C3F]
	(No symbol) [0x00007FF7CB34F6E4]
	(No symbol) [0x00007FF7CB34F87D]
	(No symbol) [0x00007FF7CB33ED49]
	BaseThreadInitThunk [0x00007FFA7E25259D+29]
	RtlUserThreadStart [0x00007FFA7F54AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CB3DFB05+28789]
	(No symbol) [0x00007FF7CB3486E0]
	(No symbol) [0

In [8]:
#6. NORTH BENGAL STATE TRANSPORT CORPORATION Bus Routes & Timings
 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/travels/nbstc"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('NBSTC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'NBSTC_bus_details.csv'")
    finally:
        driver.quit()


View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CB3DFB05+28789]
	(No symbol) [0x00007FF7CB3486E0]
	(No symbol) [0x00007FF7CB1E592A]
	(No symbol) [0x00007FF7CB23930E]
	(No symbol) [0x00007FF7CB2395FC]
	(No symbol) [0x00007FF7CB2828A7]
	(No symbol) [0x00007FF7CB25F47F]
	(No symbol) [0x00007FF7CB27F654]
	(No symbol) [0x00007FF7CB25F1E3]
	(No symbol) [0x00007FF7CB22A938]
	(No symbol) [0x00007FF7CB22BAA1]
	GetHandleVerifier [0x00007FF7CB71933D+3410093]
	GetHandleVerifier [0x00007FF7CB72E7DD+3497293]
	GetHandleVerifier [0x00007FF7CB722A73+3448803]
	GetHandleVerifier [0x00007FF7CB4A7BBB+848171]
	(No symbol) [0x00007FF7CB353C3F]
	(No symbol) [0x00007FF7CB34F6E4]
	(No symbol) [0x00007FF7CB34F87D]
	(No symbol) [0x00007FF7CB33ED49]
	BaseThreadInitThunk [0x00007FFA7E25259D+29]
	RtlUserThreadStart [0x00007FFA7F54AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CB3DFB05+28789]
	(No symbol) [0x00007FF7CB3486E0]
	(No symbol) [0

In [ ]:
#7. Kadamba Transport Corporation Limited (KTCL)
 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('KTCL_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'KTCL_bus_details.csv'")
    finally:
        driver.quit()


In [1]:
#8. PEPSU (Punjab)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('PEPSU_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'PEPSU_bus_details.csv'")
    finally:
        driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/amritsar-to-delhi-airport: Message: stale element reference: stale element not found
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D01CFB05+28789]
	(No symbol) [0x00007FF7D01386E0]
	(No symbol) [0x00007FF7CFFD592A]
	(No symbol) [0x00007FF7CFFE9F14]
	(No symbol) [0x00007FF7CFFE89C4]
	(No symbol) [0x00007FF7CFFDD109]
	(No symbol) [0x00007FF7CFFDAF52]
	(No symbol) [0x00007FF7CFFDF13C]
	(No symbol) [0x00007FF7CFFDF20F]
	(No symbol) [0x00007FF7D002326B]
	(No symbol) [0x00007FF7D004F43A]
	(No symbol) [0x00007FF7D001C5B6]
	(No symbol) [0x00007FF7D004F650]
	(No symbol) [0x00007FF7D006F654]
	(No symbol) [0x00007FF7D004F1E3]
	(No symbol) [0x00007FF7D001A938]
	(No symbol) [0x00007FF7D001BAA1]
	GetHandleVerifier [0x00007FF7D050933D+

In [2]:
# 9. KAAC TRANSPORT

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/kaac-transport"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('KAAC_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'KAAC_bus_details.csv'")
    finally:
        driver.quit()


View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7D01CFB05+28789]
	(No symbol) [0x00007FF7D01386E0]
	(No symbol) [0x00007FF7CFFD592A]
	(No symbol) [0x00007FF7D002930E]
	(No symbol) [0x00007FF7D00295FC]
	(No symbol) [0x00007FF7D00728A7]
	(No symbol) [0x00007FF7D004F47F]
	(No symbol) [0x00007FF7D006F654]
	(No symbol) [0x00007FF7D004F1E3]
	(No symbol) [0x00007FF7D001A938]
	(No symbol) [0x00007FF7D001BAA1]
	GetHandleVerifier [0x00007FF7D050933D+3410093]
	GetHandleVerifier [0x00007FF7D051E7DD+3497293]
	GetHandleVerifier [0x00007FF7D0512A73+3448803]
	GetHandleVerifier [0x00007FF7D0297BBB+848171]
	(No symbol) [0x00007FF7D0143C3F]
	(No symbol) [0x00007FF7D013F6E4]
	(No symbol) [0x00007FF7D013F87D]
	(No symbol) [0x00007FF7D012ED49]
	BaseThreadInitThunk [0x00007FFDE6AA259D+29]
	RtlUserThreadStart [0x00007FFDE760AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7D01CFB05+28789]
	(No symbol) [0x00007FF7D01386E0]
	(No symbol) [0

In [3]:
# 10. Sikkim Nationalised Transport (SNT)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


# Initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver


# Load the page
def load_page(driver, URL):
    driver.get(URL)
    time.sleep(5)


# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []


# Scrape bus details
def scrape_bus_details(driver, URL, Route):
    try:
        driver.get(URL)
        time.sleep(5)

        # Click the "View Buses" button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        # Scroll to the bottom of the page
        scrolling = True
        last_height = driver.execute_script("return document.body.scrollHeight")
        while scrolling:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scrolling = False
            last_height = new_height

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": Route,
                "Route_Link": URL,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text if i < len(price_elements) else '',
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {URL}: {e}")
        return []


# Scrape all pages
def scrape_all_pages(driver, base_url):
    all_bus_details = []
    for page in range(1, 6):
        try:
            load_page(driver, base_url)
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {e}")
    return all_bus_details


# Main script execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('SNT_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'SNT_bus_details.csv'")
    finally:
        driver.quit()


View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7D01CFB05+28789]
	(No symbol) [0x00007FF7D01386E0]
	(No symbol) [0x00007FF7CFFD592A]
	(No symbol) [0x00007FF7D002930E]
	(No symbol) [0x00007FF7D00295FC]
	(No symbol) [0x00007FF7D00728A7]
	(No symbol) [0x00007FF7D004F47F]
	(No symbol) [0x00007FF7D006F654]
	(No symbol) [0x00007FF7D004F1E3]
	(No symbol) [0x00007FF7D001A938]
	(No symbol) [0x00007FF7D001BAA1]
	GetHandleVerifier [0x00007FF7D050933D+3410093]
	GetHandleVerifier [0x00007FF7D051E7DD+3497293]
	GetHandleVerifier [0x00007FF7D0512A73+3448803]
	GetHandleVerifier [0x00007FF7D0297BBB+848171]
	(No symbol) [0x00007FF7D0143C3F]
	(No symbol) [0x00007FF7D013F6E4]
	(No symbol) [0x00007FF7D013F87D]
	(No symbol) [0x00007FF7D012ED49]
	BaseThreadInitThunk [0x00007FFDE6AA259D+29]
	RtlUserThreadStart [0x00007FFDE760AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7D01CFB05+28789]
	(No symbol) [0x00007FF7D01386E0]
	(No symbol) [0